### List datasets that will be used for training with scBasset

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
# maybe anndata and others required
# !pip install anndata h5py


In [3]:
print('here...')

here...


In [4]:
import scbasset
import anndata

2024-05-13 01:16:23.010656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-13 01:16:23.189764: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 01:16:23.194269: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-05-13 01:16:23.194288: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [5]:
print('here...')

here...


In [6]:
import os
d = '/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/theislab/mubind-pipeline/notebooks/pipeline/01_train_n_predict'
os.chdir(d)

In [7]:
import utils
import glob


### Load the datasets used in the core benchmark

In [8]:
path_by_dataset = utils.get_datasets()
path_by_dataset


{'organoids': '/mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks_obs2000_var4000.h5ad',
 'gbm': '/mnt/f/workspace/theislab/mubind/data/gbm_multiome_scdori/23_01_23_atac_compressed_n1000.h5ad',
 'noack_2022': '/mnt/f/workspace/theislab/mubind/data/noack_2022/*/merged_scATAC_integrated_cicero_faye_chong_obs*_var*.h5ad',
 'pancreatic_endocrinogenesis': '/mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/*/pancreas_multiome_2022_processed_atac_obs*_var*.h5ad',
 'pbmc': '/mnt/f/workspace/theislab/mubind/data/pbmc/*/processed_laura_2023_obs*_var*.h5ad'}

In [9]:
import os
os.chdir('/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/scbasset/bin')

## Run python using the scbasset environment. If not found, please set up manually

In [10]:
pythonbin = '/home/ilibarra/.conda/envs/scbasset/bin/python' # ''

In [11]:
# adata = anndata.read_h5ad(ad_path)
# print(adata.shape)
# adata.var['chr'] = adata.var_names.str.split('-').str[0]
# adata.var['chr'] = np.where(~adata.var['chr'].str.contains('chr'), 'chr', '') + adata.var['chr']
# adata.var['start'] = adata.var_names.str.split('-').str[1]
# adata.var['end'] = adata.var_names.str.split('-').str[2]
# adata.write(ad_path, compression='lzf')
# genome_fasta = '/mnt/c/Users/ignacio.ibarra/Dropbox/annotations/%s/genome/%s.fa' % (species, species)
# os.path.exists(genome_fasta)
# print(genome_fasta)


In [12]:
for k in path_by_dataset:
    print(k, path_by_dataset[k])

organoids /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks_obs2000_var4000.h5ad
gbm /mnt/f/workspace/theislab/mubind/data/gbm_multiome_scdori/23_01_23_atac_compressed_n1000.h5ad
noack_2022 /mnt/f/workspace/theislab/mubind/data/noack_2022/*/merged_scATAC_integrated_cicero_faye_chong_obs*_var*.h5ad
pancreatic_endocrinogenesis /mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/*/pancreas_multiome_2022_processed_atac_obs*_var*.h5ad
pbmc /mnt/f/workspace/theislab/mubind/data/pbmc/*/processed_laura_2023_obs*_var*.h5ad


In [13]:
import datetime
import pickle

In [14]:
import os
import anndata
import os
import numpy as np

overwrite_prepare = False
overwrite_train = True
query_epochs = [1, 10, 20, 50, 100] #  5, 10, 50, 100]

genomes_path = '/mnt/c/Users/IgnacioIbarra/Dropbox/annotations/%s/genome/%s.fa'

for k in path_by_dataset:
    try:
        print(k)
        # if not 'noack_2022' in k:
        #     continue
        for n_epochs in query_epochs:
            for ad_path in glob.glob(path_by_dataset[k]):
                if 'ALL' in ad_path: #  and not '_obs500_' in ad_path: #  or not '/random/' in ad_path:
                    continue
                print(os.path.exists(ad_path), ad_path)
                # print('\nnext path')
                print(k, ad_path)
                n_cells = ad_path.split('_')[-2]
                sampling_method = ad_path.split('/')[-2]

                print(k, sampling_method, ad_path)

                species = 'hg38' if (k != 'noack_2022' and k != 'pancreatic_endocrinogenesis') else 'mm10'

                filename = os.path.basename(ad_path)
                input_dir = os.path.join(ad_path.replace(filename, 'scbasset_input'), n_cells)


                for use_poisson in [1, 0]:
                    out_dir = os.path.join(ad_path.replace(filename,
                                                        'scbasset_output'),
                                                        n_cells + '_e%i' % n_epochs, 'poisson' if use_poisson else 'bce')
                    
                    print('in', input_dir)
                    print('out', os.path.exists(out_dir), out_dir)
                    # assert False

                    if not os.path.exists(input_dir):
                        os.makedirs(input_dir)
                    if not os.path.exists(out_dir):
                        os.makedirs(out_dir)
                    
                    print(species)
                    train_path = os.path.join(input_dir, 'ad.h5ad')
                    train_seqs_path = os.path.join(input_dir, 'train_seqs.h5')

                    print(os.path.exists(train_path), train_path)
                    print(os.path.exists(train_seqs_path), train_seqs_path)
                    # assert False

                    outpath_time = os.path.join(out_dir, 'running_time.pkl')

                    if not os.path.exists(train_path) or not os.path.exists(train_seqs_path) or not os.path.exists(outpath_time) or overwrite_prepare:
                        print('preparing data using path....')
                        print(ad_path)
                        adata = anndata.read_h5ad(ad_path)
                        print(adata.shape)

                        # assert False
                        adata.var['chr'] = np.where(adata.var_names.str.contains(':'), adata.var_names.str.split(':').str[0], adata.var_names.str.split('-').str[0])
                        adata.var['chr'] = np.where(~adata.var['chr'].str.contains('chr'), 'chr', '') + adata.var['chr']
                        adata.var['start'] = adata.var_names.str.split('-').str[1]
                        adata.var['end'] = np.where(~adata.var_names.str.contains(':'), adata.var_names.str.split('-').str[2], adata.var_names.str.split('-').str[1])
                        adata.write(ad_path, compression='lzf')
                        genome_fasta = genomes_path % (species, species)
                        os.path.exists(genome_fasta)
                        print(genome_fasta)
                        # preprocess fasta
                        print('')
                        !echo $pythonbin scbasset_preprocess.py --ad_file $ad_path --input_fasta $genome_fasta --out_path $input_dir --batch 50
                        !$pythonbin scbasset_preprocess.py --ad_file $ad_path --input_fasta $genome_fasta --out_path $input_dir --batch 50
                        print('')
                    else:
                        print('skip prepare (already done...)')
                    
                    best_model_path = os.path.join(out_dir, 'best_model.h5')
                    if not os.path.exists(best_model_path) or overwrite_train:
                        print('')

                        !echo $pythonbin scbasset_train.py --input_folder $input_dir --epochs $n_epochs --out_path $out_dir
                        print('about to train...')

                        t1 = datetime.datetime.now()
                        # assert False
                        !$pythonbin scbasset_train.py --input_folder $input_dir --epochs $n_epochs --out_path $out_dir --use_poisson $use_poisson
                        print('')
                        t2 = datetime.datetime.now()
                        time_diff = (t2 - t1)

                        pickle.dump(outpath_time, open(outpath_time, 'wb'))
                        print(os.path.exists(outpath_time), outpath_time)
                        # assert False
                    else:
                        print('skip train (already done...)')
                    print('')


    except Exception:
        pass

    print('')

organoids
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks_obs2000_var4000.h5ad
organoids /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks_obs2000_var4000.h5ad
organoids organoids_treutlein_dataset /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/RNA_ATAC_metacells_sce_peaks_obs2000_var4000.h5ad
in /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/scbasset_input/obs2000
out True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/scbasset_output/obs2000_e1/poisson
hg38
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/scbasset_input/obs2000/ad.h5ad
True /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_dataset/scbasset_input/obs2000/train_seqs.h5
skip prepare (already done...)

/home/ilibarra/.conda/envs/scbasset/bin/python scbasset_train.py --input_folder /mnt/f/workspace/theislab/mubind/data/organoids_treutlein_da

process 1000 peaks takes 2.9 s
^C
Traceback (most recent call last):
  File "scbasset_preprocess.py", line 68, in <module>
    main()
  File "scbasset_preprocess.py", line 58, in main
    make_h5_sparse(ad, '%s/all_seqs.h5'%output_path, input_fasta, batch_size=args.batch)
  File "/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/scBasset/scbasset/utils.py", line 167, in make_h5_sparse
    dna_array_dense = [dna_1hot_2vec(x) for x in seqs_dna]
  File "/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/scBasset/scbasset/utils.py", line 167, in <listcomp>
    dna_array_dense = [dna_1hot_2vec(x) for x in seqs_dna]
  File "/mnt/c/Users/IgnacioIbarra/Dropbox/workspace/scBasset/scbasset/utils.py", line 100, in dna_1hot_2vec
    nt = seq[i - seq_start]
KeyboardInterrupt


/home/ilibarra/.conda/envs/scbasset/bin/python scbasset_train.py --input_folder /mnt/f/workspace/theislab/mubind/data/pancreatic_endocrinogenesis/random/scbasset_input/obs5000 --epochs 10 --out_path /mnt/f/workspace/theislab/mubind/data